# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [476]:
problem = '''The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits.'''

## 2. Generate the mathematical model

In [482]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [483]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for PRODA, S.A. Weekly Production Scheduling

**Sets:**

* $P = \{P1, P2, P3\}$: Set of products.

**Parameters:**

* $SP_p$: Unit sales price of product $p \in P$. 
    * $SP_{P1} = 1.50$
    * $SP_{P2} = 1.80$
    * $SP_{P3} = 1.97$
* $CS_p$: Unit cost of smelting product $p \in P$ at PRODA.
    * $CS_{P1} = 0.30$
    * $CS_{P2} = 0.50$
    * $CS_{P3} = 0.40$
* $CSS_p$: Unit cost of subcontracted smelting for product $p \in \{P1, P2\}$.
    * $CSS_{P1} = 0.50$
    * $CSS_{P2} = 0.60$
* $CM_p$: Unit cost of mechanisation for product $p \in P$.
    * $CM_{P1} = 0.20$
    * $CM_{P2} = 0.10$
    * $CM_{P3} = 0.27$
* $CAP_p$: Unit cost of assembly and packaging for product $p \in P$.
    * $CAP_{P1} = 0.30$
    * $CAP_{P2} = 0.20$
    * $CAP_{P3} = 0.20$
* $TS_p$: Smelting time (in minutes) per unit of product $p \in P$ at PRODA.
    * $TS_{P1} = 6$
    * $TS_{P2} = 10$
    * $TS_{P3} = 8$
* $TM_p$: Mechanisation time (in minutes) per unit of product $p \in P$.
    * $TM_{P1} = 6$
    * $TM_{P2} = 3$
    * $TM_{P3} = 8$
* $TAP_p$: Assembly and packaging time (in minutes) per unit of product $p \in P$.
    * $TAP_{P1} = 3$
    * $TAP_{P2} = 2$
    * $TAP_{P3} = 2$
* $CapS = 8000$: Total available smelting time (in minutes) at PRODA per week.
* $CapM = 12000$: Total available mechanisation time (in minutes) per week.
* $CapAP = 10000$: Total available assembly and packaging time (in minutes) per week.

**Decision Variables:**

* $X_p$: Quantity of product $p \in P$ to produce per week.
* $XS_p$: Quantity of product $p \in \{P1, P2\}$ to subcontract smelting per week.

**Objective Function:**

Maximize weekly profit:

```
Maximize Z = Σ_{p∈P} (SP_p * X_p - CS_p * (X_p - XS_p) - CSS_p * XS_p - CM_p * X_p - CAP_p * X_p) 
```

**Constraints:**

1. **Smelting capacity at PRODA:** 
    *  Σ_{p∈P} TS_p * (X_p - XS_p) ≤ CapS
2. **Mechanisation capacity:**
    * Σ_{p∈P} TM_p * X_p ≤ CapM
3. **Assembly and packaging capacity:**
    * Σ_{p∈P} TAP_p * X_p ≤ CapAP
4. **Non-negativity:**
    *  X_p ≥ 0,  ∀p ∈ P 
    *  XS_p ≥ 0, ∀p ∈ {P1, P2}
5. **Subcontracting limit:**
    *  XS_p ≤ X_p, ∀p ∈ {P1, P2}

This model can be solved using linear programming solvers to determine the optimal production quantities for each product and the extent of subcontracting for smelting to maximize PRODA's weekly profit. 


## 3. Generate the pyomo code

In [484]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [485]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define sets
products = ["P1", "P2", "P3"]

# Define parameters
SP = {"P1": 1.50, "P2": 1.80, "P3": 1.97}  
CS = {"P1": 0.30, "P2": 0.50, "P3": 0.40} 
CSS = {"P1": 0.50, "P2": 0.60} 
CM = {"P1": 0.20, "P2": 0.10, "P3": 0.27} 
CAP = {"P1": 0.30, "P2": 0.20, "P3": 0.20} 
TS = {"P1": 6, "P2": 10, "P3": 8} 
TM = {"P1": 6, "P2": 3, "P3": 8} 
TAP = {"P1": 3, "P2": 2, "P3": 2}
CapS = 8000
CapM = 12000
CapAP = 10000

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.X = pyo.Var(products, domain=pyo.NonNegativeReals)
model.XS = pyo.Var(["P1", "P2"], domain=pyo.NonNegativeReals)

# Define objective function
model.profit = pyo.Objective(
    expr=sum(SP[p] * model.X[p] - CS[p] * (model.X[p] - model.XS[p]) 
              - CSS.get(p, 0) * model.XS[p] - CM[p] * model.X[p] - CAP[p] * model.X[p] 
              for p in products),
    sense=pyo.maximize
)

# Define constraints
model.smelting_capacity = pyo.Constraint(
    expr=sum(TS[p] * (model.X[p] - model.XS.get(p, 0)) for p in products) <= CapS
)

model.mechanisation_capacity = pyo.Constraint(
    expr=sum(TM[p] * model.X[p] for p in products) <= CapM
)

model.assembly_capacity = pyo.Constraint(
    expr=sum(TAP[p] * model.X[p] for p in products) <= CapAP
)

model.subcontracting_limit = pyo.ConstraintList()
for p in ["P1", "P2"]:
    model.subcontracting_limit.add(expr=model.XS[p] <= model.X[p])

# Solve model
solver = pyo.SolverFactory('glpk')  # You can change the solver if desired
results = solver.solve(model)

# Print results
print(results)
print("Optimal solution:")
for p in products:
    print(f"Produce {model.X[p].value:.2f} units of {p}")
for p in ["P1", "P2"]:
    print(f"Subcontract {model.XS[p].value:.2f} units of {p} for smelting")
print(f"Total profit: {model.profit():.2f}")
```

This code defines the sets, parameters, variables, objective function, and constraints for the given mathematical model. Sample data is included in the code for the parameters. The code then uses the GLPK solver to solve the model and outputs the optimal production quantities for each product, the extent of subcontracting for smelting, and the total profit. 


## 4. Input problem data and try running the generated code

In [486]:
import pyomo.environ as pyo

# Define sets
products = ["P1", "P2", "P3"]

# Define parameters
SP = {"P1": 1.50, "P2": 1.80, "P3": 1.97}  
CS = {"P1": 0.30, "P2": 0.50, "P3": 0.40} 
CSS = {"P1": 0.50, "P2": 0.60} 
CM = {"P1": 0.20, "P2": 0.10, "P3": 0.27} 
CAP = {"P1": 0.30, "P2": 0.20, "P3": 0.20} 
TS = {"P1": 6, "P2": 10, "P3": 8} 
TM = {"P1": 6, "P2": 3, "P3": 8} 
TAP = {"P1": 3, "P2": 2, "P3": 2}
CapS = 8000
CapM = 12000
CapAP = 10000

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.X = pyo.Var(products, domain=pyo.NonNegativeReals)
model.XS = pyo.Var(["P1", "P2"], domain=pyo.NonNegativeReals)

# Define objective function
model.profit = pyo.Objective(
    expr=sum(SP[p] * model.X[p] - CS[p] * (model.X[p] - model.XS[p]) 
              - CSS.get(p, 0) * model.XS[p] - CM[p] * model.X[p] - CAP[p] * model.X[p] 
              for p in products),
    sense=pyo.maximize
)

# Define constraints
model.smelting_capacity = pyo.Constraint(
    expr=sum(TS[p] * (model.X[p] - model.XS.get(p, 0)) for p in products) <= CapS
)

model.mechanisation_capacity = pyo.Constraint(
    expr=sum(TM[p] * model.X[p] for p in products) <= CapM
)

model.assembly_capacity = pyo.Constraint(
    expr=sum(TAP[p] * model.X[p] for p in products) <= CapAP
)

model.subcontracting_limit = pyo.ConstraintList()
for p in ["P1", "P2"]:
    model.subcontracting_limit.add(expr=model.XS[p] <= model.X[p])

# Solve model
solver = pyo.SolverFactory('glpk')  # You can change the solver if desired
results = solver.solve(model)

# Print results
print(results)
print("Optimal solution:")
for p in products:
    print(f"Produce {model.X[p].value:.2f} units of {p}")
for p in ["P1", "P2"]:
    print(f"Subcontract {model.XS[p].value:.2f} units of {p} for smelting")
print(f"Total profit: {model.profit():.2f}")

KeyError: "Index 'P3' is not valid for indexed component 'XS'"

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [487]:
print(response.text)

## Mathematical Optimization Model for PRODA, S.A. Weekly Production Scheduling

**Sets:**

* $P = \{P1, P2, P3\}$: Set of products.

**Parameters:**

* $SP_p$: Unit sales price of product $p \in P$. 
    * $SP_{P1} = 1.50$
    * $SP_{P2} = 1.80$
    * $SP_{P3} = 1.97$
* $CS_p$: Unit cost of smelting product $p \in P$ at PRODA.
    * $CS_{P1} = 0.30$
    * $CS_{P2} = 0.50$
    * $CS_{P3} = 0.40$
* $CSS_p$: Unit cost of subcontracted smelting for product $p \in \{P1, P2\}$.
    * $CSS_{P1} = 0.50$
    * $CSS_{P2} = 0.60$
* $CM_p$: Unit cost of mechanisation for product $p \in P$.
    * $CM_{P1} = 0.20$
    * $CM_{P2} = 0.10$
    * $CM_{P3} = 0.27$
* $CAP_p$: Unit cost of assembly and packaging for product $p \in P$.
    * $CAP_{P1} = 0.30$
    * $CAP_{P2} = 0.20$
    * $CAP_{P3} = 0.20$
* $TS_p$: Smelting time (in minutes) per unit of product $p \in P$ at PRODA.
    * $TS_{P1} = 6$
    * $TS_{P2} = 10$
    * $TS_{P3} = 8$
* $TM_p$: Mechanisation time (in minutes) per unit of product $p 

In [489]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define sets
products = ["P1", "P2", "P3"]

# Define parameters
SP = {"P1": 1.50, "P2": 1.80, "P3": 1.97}  
CS = {"P1": 0.30, "P2": 0.50, "P3": 0.40} 
CSS = {"P1": 0.50, "P2": 0.60} 
CM = {"P1": 0.20, "P2": 0.10, "P3": 0.27} 
CAP = {"P1": 0.30, "P2": 0.20, "P3": 0.20} 
TS = {"P1": 6, "P2": 10, "P3": 8} 
TM = {"P1": 6, "P2": 3, "P3": 8} 
TAP = {"P1": 3, "P2": 2, "P3": 2}
CapS = 8000
CapM = 12000
CapAP = 10000

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.X = pyo.Var(products, domain=pyo.NonNegativeReals)
model.XS = pyo.Var(["P1", "P2"], domain=pyo.NonNegativeReals)

# Define objective function
model.profit = pyo.Objective(
    expr=sum(SP[p] * model.X[p] - CS[p] * (model.X[p] - model.XS[p]) 
              - CSS.get(p, 0) * model.XS[p] - CM[p] * model.X[p] - CAP[p] * model.X[p] 
              for p in products),
    sense=pyo.maximize
)

# Define constraints
model.smelting_capacity = pyo.Constraint(
    expr=sum(TS[